# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torchdata.datapipes as dp
import torchtext.transforms as T
import spacy
from torchtext.vocab import build_vocab_from_iterator
eng = spacy.load("en_core_web_sm")

In [ ]:
import pandas as pd

evidences = pd.read_json('/content/drive/MyDrive/nlp/data/evidence.json', orient='index')
train_claims = pd.read_json('/content/drive/MyDrive/nlp/data/train-claims.json', orient='index')
dev_claims = pd.read_json('/content/drive/MyDrive/nlp/data/dev-claims.json', orient='index')
#test_claims = pd.read_json('/content/drive/MyDrive/nlp/data/test-claims-unlabelled.json', orient='index')

#update column names
evidences.reset_index(inplace=True)
evidences.columns = ['evidence_id', 'evidence_text']

train_claims.reset_index(inplace=True)
train_claims.rename(columns={'index': 'claim_id'}, inplace=True)

dev_claims.reset_index(inplace=True)
dev_claims.rename(columns={'index': 'claim_id'}, inplace=True)

evidence_id = evidences['evidence_id']
evidence_text = evidences['evidence_text']
evidence_idx = evidences.index.tolist()

evidence_id_dict = dict(zip(evidence_id, evidence_idx))

train_claims_text = train_claims['claim_text']
train_evidence_ids = train_claims['evidences']
#map evidence_id to their corrosponding index for faster processing
train_evidence_idxs = train_evidence_ids.apply(lambda x: [evidence_id_dict[evidence_id] for evidence_id in x])

dev_claims_text = dev_claims['claim_text']
dev_evidence_ids = dev_claims['evidences']
dev_evidence_idxs = dev_evidence_ids.apply(lambda x: [evidence_id_dict[evidence_id] for evidence_id in x])

In [ ]:
#text preprocessing
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))
#lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# def preprocess_data(text):
#     tokens = tt.tokenize(text)

#     processed_tokens = []

#     for token in tokens:
#         token = token.lower()
#         if token not in stopwords and token.isalpha():
#             lemma = lemmatizer.lemmatize(token)
#             processed_tokens.append(lemma)

#     return processed_tokens

def preprocess_data(text):
    tokens = tt.tokenize(text)

    processed_tokens = []

    for token in tokens:
        token = token.lower()
        if token not in stopwords and token.isalpha():
            stem = stemmer.stem(token)
            processed_tokens.append(stem)

    return processed_tokens

train_claims_text_processed = train_claims_text.apply(preprocess_data)
dev_claims_text_precessed = dev_claims_text.apply(preprocess_data)
evidence_text_processed = evidence_text.apply(preprocess_data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import json

train_claims_text_processed = json.load(open("/content/drive/MyDrive/nlp/data/train_claims_text_processed.json", "r"))
dev_claims_text_precessed = json.load(open("/content/drive/MyDrive/nlp/data/dev_claims_text_precessed.json", "r"))
evidence_text_processed = json.load(open("/content/drive/MyDrive/nlp/data/evidence_text_processed.json", "r"))

In [ ]:
from gensim.models import Word2Vec

word2vec_model = Word2Vec.load('/content/drive/MyDrive/nlp/data/word2vec_model.bin')

import numpy as np
def generate_embedding(text, model):
    # Filter out words that are not in the Word2Vec model's vocabulary
    words = [word for word in text if word in model.wv.key_to_index]
    if not words:  # Handle cases where none of the words are in the vocabulary
        return np.zeros(model.vector_size)
    # Get embeddings for each word in the text and average them
    word_embeddings = [model.wv[word] for word in words]
    sentence_embedding = np.mean(word_embeddings, axis=0)
    return sentence_embedding

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Generate embeddings for train claim_text
train_claim_text_embeddings = [generate_embedding(text, word2vec_model) for text in train_claims_text_processed]

# Generate embeddings for dev claim_text
dev_claim_text_embeddings = [generate_embedding(text, word2vec_model) for text in dev_claims_text_precessed]

# Generate embeddings for all evidence texts
evidence_embeddings = [generate_embedding(text, word2vec_model) for text in evidence_text_processed]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to compute cosine similarity scores for all claims and evidence embeddings
def compute_similarity_scores(claim_embeddings, evidence_embeddings):
    similarity_scores = cosine_similarity(claim_embeddings, evidence_embeddings)
    return similarity_scores


# Compute cosine similarity scores for training claims and evidence embeddings
train_similarity_scores = compute_similarity_scores(train_claim_text_embeddings, evidence_embeddings)
dev_similarity_scores = compute_similarity_scores(dev_claim_text_embeddings, evidence_embeddings)

NameError: name 'train_claim_text_embeddings' is not defined

In [ ]:
import torch
def get_top_k_indices(similarity_scores, k=1000):
    # Args:
    # similarity_scores: a PyTorch tensor of shape (num_claims, num_evidences)
    # Returns:
    # top_k_indices: a tensor of shape (num_claims, k) containing indices of the top k elements

    # Convert numpy array to PyTorch tensor if needed
    if isinstance(similarity_scores, np.ndarray):
        similarity_scores_tensor = torch.FloatTensor(similarity_scores)
    else:
        similarity_scores_tensor = similarity_scores

    # Get top k indices for each sample using PyTorch
    top_k_values, top_k_indices = torch.topk(similarity_scores_tensor, k, dim=1, largest=True, sorted=True)

    return top_k_indices,top_k_values

# Compute top 1000 indices for training and development sets
train_top_indices, _ = get_top_k_indices(train_similarity_scores, k=500)
dev_top_indices, dev_orig_scores = get_top_k_indices(dev_similarity_scores, k=20)

In [ ]:
from collections import defaultdict
import numpy as np
from scipy.sparse import csr_matrix, diags

# Build inverted index
def build_inverted_index(documents):
    inverted_index = defaultdict(list)
    for i, doc in enumerate(documents):
        term_freq = defaultdict(int)
        for term in doc:
            term_freq[term] += 1
        for term, freq in term_freq.items():
            inverted_index[term].append((i, freq))
    return inverted_index

# Compute IDF values
def compute_idf_values(inverted_index, total_documents):
    idf_values = {}
    for term, postings in inverted_index.items():
        idf_values[term] = np.log((total_documents + 1) / (len(postings) + 1))
    return idf_values

# Calculate average document length
def calculate_avg_doc_length(documents):
    total_length = sum(len(doc) for doc in documents)
    return total_length / len(documents)
#k.1.0, b = 0.78 recall=0.14
# Compute BM25 scores
def bm25_scores(query, inverted_index, idf_values, avg_doc_length, k1=0.4, b=0.9):
    scores = defaultdict(float)
    for term in query:
        if term not in inverted_index:
            continue
        doc_list = inverted_index[term]
        idf = idf_values[term]
        for doc_id, tf in doc_list:
            # Compute BM25 score for this document
            doc_length = len(evidence_text_processed[doc_id])
            numerator = idf * tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * (doc_length / avg_doc_length))
            scores[doc_id] += numerator / denominator
    return scores

# Build inverted index for evidence text
inverted_index = build_inverted_index(evidence_text_processed)
total_documents = len(evidence_text_processed)
idf_values = compute_idf_values(inverted_index, total_documents)
avg_doc_length = calculate_avg_doc_length(evidence_text_processed)

# Example usage
train_bm25_results = []
for query in train_claims_text_processed:
    scores = bm25_scores(query, inverted_index, idf_values, avg_doc_length)
    train_bm25_results.append(scores)

dev_bm25_results = []
for query in dev_claims_text_precessed:
    scores = bm25_scores(query, inverted_index, idf_values, avg_doc_length)
    dev_bm25_results.append(scores)



In [ ]:
train_reranked_indices = [[doc_id for doc_id, _ in sorted(scores.items(), key=lambda x: x[1], reverse=True)] for scores in train_bm25_results]
train_reranked_scores = [[score for _, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)] for scores in train_bm25_results]

dev_reranked_indices = [[doc_id for doc_id, _ in sorted(scores.items(), key=lambda x: x[1], reverse=True)] for scores in dev_bm25_results]
dev_reranked_scores = [[score for _, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)] for scores in dev_bm25_results]

In [ ]:
def topk_indices(indices, k=100):
    return [indices[i][:min(k, len(indices[i]))] for i in range(len(indices))]

train_top_indices = topk_indices(train_reranked_indices, k=50)
dev_top_indices = topk_indices(dev_reranked_indices, k=50)
dev_top_scores = topk_indices(dev_reranked_scores, k=50)

In [ ]:
def calculate_average_recall(top_k_indices, true_indices):
    """
    Calculates the average recall for each claim based on the overlap between the top-k indices and the true indices.

    Args:
    - top_k_indices (List[List[int]]): List of lists, where each sublist contains the top k indices for each claim.
    - true_indices (List[List[int]]): List of lists, where each sublist contains the actual true indices for each claim.

    Returns:
    - float: Average recall across all claims.
    """
    recall_values = []

    # Iterate over each pair of top_k_indices and corresponding true indices
    for top_indices, true_inds in zip(top_k_indices, true_indices):
        # Calculate the number of true positives
        true_positives = len(set(top_indices) & set(true_inds))

        # Calculate recall for this claim
        recall = true_positives / len(true_inds) if true_inds else 0  # Avoid division by zero

        # Append the recall for this claim to the list
        recall_values.append(recall)

    # Compute average recall over all claims
    avg_recall = sum(recall_values) / len(recall_values) if recall_values else 0  # Avoid division by zero if list is empty

    return avg_recall

avg_recall = calculate_average_recall(dev_top_indices, dev_evidence_idxs)
print("Average Recall:", avg_recall)

Average Recall: 0.8329004329004328


In [ ]:
from collections import Counter, OrderedDict
def build_vocab(texts, min_freq=3):
    # Count all the words
    word_freq = Counter()
    for text in texts:
        word_freq.update(text)

    # Start vocab from special tokens
    vocab = OrderedDict({
        "<pad>": 0,
        "<unk>": 1,
        "<sos>": 2,
        "<eos>": 3
    })
    index = 4  # Start indexing from 4 because 0-3 are reserved for special tokens
    for word, freq in word_freq.items():
        if freq >= min_freq:  # Only include words that meet the frequency threshold
            vocab[word] = index
            index += 1

    return vocab

# Build vocabulary using only evidence texts and applying the frequency threshold
vocab = build_vocab(evidence_text_processed, min_freq=3)


def numericalize(text, vocab):
    # Convert text to numerical representation, adding start and end tokens, and padding
    numericalized = [vocab["<sos>"]] + [vocab.get(word, vocab["<unk>"]) for word in text] + [vocab["<eos>"]]
    #padded_numericalized = numericalized[:max_length] + [0] * max(0, max_length - len(numericalized))
    return numericalized

train_claims_numerical = [numericalize(x, vocab) for x in train_claims_text_processed]
dev_claims_numerical = [numericalize(x, vocab) for x in dev_claims_text_precessed]
evidence_numerical = [numericalize(x, vocab) for x in evidence_text_processed]


In [ ]:
def build_vocab(texts, min_freq=3):
    word_freq = Counter()
    for text in texts:
        word_freq.update(text)

    vocab = OrderedDict({
        "<pad>": 0,
        "<unk>": 1,
        "<cls>": 2,
        "<sep>": 3# Adding CLS token
    })
    index = 4  # Start indexing from 3 because 0-2 are reserved for special tokens
    for word, freq in word_freq.items():
        if freq >= min_freq:
            vocab[word] = index
            index += 1

    return vocab

vocab = build_vocab(evidence_text_processed, min_freq=3)

## 1.1 Training DataLoading

In [ ]:
import random
from torch.utils.data import Dataset

class RankingDataset(Dataset):
    def __init__(self, claims, evidences, true_indices, bm25_indices):
        """
        Dataset for claim-evidence ranking with in-batch negatives handled during training.
        :param claims: List of pre-padded and numericalized claims.
        :param evidences: List of pre-padded and numericalized evidences.
        :param true_indices: List of lists, each sublist contains indices of true evidences for the corresponding claim.
        :param bm25_indices: List of lists, each sublist contains BM25-ranked evidence indices for the corresponding claim.
        """
        self.claims = claims
        self.evidences = evidences
        self.true_indices = true_indices
        self.bm25_indices = bm25_indices

    def __len__(self):
        return len(self.claims)

    def __getitem__(self, idx):
        claim = self.claims[idx]
        #candidate_pos_indices = [i for i in self.true_indices[idx] if i in self.bm25_indices[idx]]

        # if not candidate_pos_indices:
        #     #print(f"No true evidences found for claim {claim}")
        #     return None  # Skip if no true index is in the top BM25 results

        candidate_pos_indices = self.true_indices[idx]
        pos_idx = random.choice(candidate_pos_indices)  # Randomly choose one positive evidence from candidates
        positive_evidence = self.evidences[pos_idx]

        bm25_top_5 = [i for i in self.bm25_indices[idx] if i not in self.true_indices[idx]][:20]
        if not bm25_top_5:
            return None  # Skip if no BM25 negatives in the top 5

        neg_idx = random.choice(bm25_top_5)  # Randomly choose one negative evidence from BM25 top 5
        bm25_negative = self.evidences[neg_idx]

        return claim, positive_evidence, bm25_negative



def text_to_indices(text, vocab):
    indices = [vocab["<sos>"]] + [vocab.get(word, vocab["<unk>"]) for word in text] + [vocab["<eos>"]]
    return indices

def pad_sequences(sequences, pad_idx=0):
    return pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in sequences], batch_first=True, padding_value=pad_idx)


def collate_fn(batch):
    # Filter out None entries
    batch = [item for item in batch if item is not None]

    if len(batch) == 0:
        return None, None, None, None, None, None

    claims, positive_evidences, bm25_negatives = zip(*batch)

    claims_indices = [text_to_indices(claim, vocab) for claim in claims]
    positive_evidences_indices = [text_to_indices(evidence, vocab) for evidence in positive_evidences]
    bm25_negatives_indices = [text_to_indices(evidence, vocab) for evidence in bm25_negatives]

    claims_tensor = pad_sequences(claims_indices, pad_idx=vocab["<pad>"])
    positive_evidences_tensor = pad_sequences(positive_evidences_indices, pad_idx=vocab["<pad>"])

    batch_size = len(claims)
    in_batch_positives = list(positive_evidences)
    in_batch_negatives = list(bm25_negatives)

    combined_negative_evidences = []

    for i in range(batch_size):
        # In-batch positives from other claims
        in_batch_pos_negatives = [evidence for j, evidence in enumerate(in_batch_positives) if j != i]

        # Randomly sample 7 gold negatives (positives of other claims)
        gold_negatives = random.sample(in_batch_pos_negatives, min(31, len(in_batch_pos_negatives)))

        # Randomly sample 7 BM25 negatives
        bm25_negatives_sampled = random.sample(in_batch_negatives, min(32, len(in_batch_negatives)))

        # Combine sampled gold negatives and BM25 negatives into a single list
        combined_negatives = gold_negatives + bm25_negatives_sampled
        combined_negative_texts = [text_to_indices(evidence, vocab) for evidence in combined_negatives]

        combined_negatives_tensors = pad_sequences(combined_negative_texts, pad_idx=vocab["<pad>"])
        combined_negative_evidences.append(combined_negatives_tensors)

    max_length_neg = max(neg.shape[1] for neg in combined_negative_evidences)
    combined_negative_evidences_tensor = torch.stack([F.pad(neg, (0, max_length_neg - neg.shape[1]), value=vocab["<pad>"]) for neg in combined_negative_evidences], dim=0)

    return claims_tensor, positive_evidences_tensor, combined_negative_evidences_tensor

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset

# def text_to_indices(text, vocab):
#     indices = [vocab["<cls>"]] + [vocab.get(word, vocab["<unk>"]) for word in text]
#     return indices

def text_to_indices(text, vocab):
    return [vocab.get(word, vocab["<unk>"]) for word in text]
class RankingDataset(Dataset):
    def __init__(self, claims, evidences, true_indices, top_indices, vocab):
        self.claims = claims
        self.evidences = evidences
        self.true_indices = true_indices
        self.top_indices = top_indices
        self.vocab = vocab
        self.pairs = self.create_pairs()

    def create_pairs(self):
        pairs = []
        for idx, claim in enumerate(self.claims):
            candidate_pos_indices = self.true_indices[idx]
            positive_evidences = [self.evidences[i] for i in candidate_pos_indices]

            candidate_neg_indices = [i for i in self.top_indices[idx] if i not in candidate_pos_indices]
            top_neg_evidences = [self.evidences[i] for i in candidate_neg_indices[:7]]

            for pos_evidence in positive_evidences:
                pairs.append((claim, pos_evidence, top_neg_evidences))

        return pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        claim, pos_evidence, top_neg_evidences = self.pairs[idx]
        claim_indices = text_to_indices(claim, self.vocab)
        pos_evidence_indices = text_to_indices(pos_evidence, self.vocab)
        top_neg_evidence_indices = [text_to_indices(neg, self.vocab) for neg in top_neg_evidences]

        return claim_indices, pos_evidence_indices, top_neg_evidence_indices



# def collate_fn(batch):
#     batch = [item for item in batch if item is not None]

#     if len(batch) == 0:
#         return None, None

#     sequences, labels = zip(*batch)

#     sequences_tensor = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in sequences], batch_first=True, padding_value=vocab["<pad>"]).to(device)
#     labels_tensor = torch.tensor(labels, dtype=torch.float).to(device)

#     return sequences_tensor, labels_tensor
def collate_fn(batch):
    batch = [item for item in batch if item is not None]

    if len(batch) == 0:
        return None, None

    claims, positive_evidences, top_negatives = zip(*batch)

    # Initialize tensors
    combined_sequences = []
    combined_labels = []

    batch_size = len(claims)

    # Pad and add top negatives
    for i in range(batch_size):
        claim = claims[i]
        pos_evidence = positive_evidences[i]
        top_neg_evidences = top_negatives[i]

        # Create positive pair
        combined_sequences.append([vocab["<cls>"]] + claim + [vocab["<sep>"]] + pos_evidence + [vocab["<sep>"]])
        combined_labels.append(1)

        # Add 7 top negatives
        for neg_evidence in top_neg_evidences[:4]:
            combined_sequences.append([vocab["<cls>"]] + claim + [vocab["<sep>"]] + neg_evidence + [vocab["<sep>"]])
            combined_labels.append(0)

    # Add in-batch negatives
    for i in range(batch_size):
        for j in range(batch_size):
            if i != j:
                claim = claims[i]
                pos_evidence = positive_evidences[j]

                combined_sequences.append([vocab["<cls>"]] + claim + [vocab["<sep>"]] + pos_evidence + [vocab["<sep>"]])
                combined_labels.append(0)

                if len(combined_labels) % 4 == 0:
                    break

    # Convert to tensors and pad
    sequences_tensor = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in combined_sequences], batch_first=True, padding_value=vocab["<pad>"]).to(device)
    labels_tensor = torch.tensor(combined_labels, dtype=torch.float).to(device)

    return sequences_tensor, labels_tensor



# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 2.1 Loss Functions

In [ ]:
def listwise_loss(model, claims, pos_evidences, neg_evidences, pos_weight=2.0):
    batch_size = claims.size(0)
    num_negatives = neg_evidences.shape[1]
    # Get the scores for positive evidence
    pos_scores = model(claims, pos_evidences).unsqueeze(1)
    # Get the scores for negative evidence
    neg_scores_list = [model(claims, neg_evidences[:, i, :]).unsqueeze(1) for i in range(neg_evidences.shape[1])]
    neg_scores = torch.cat(neg_scores_list, dim=1)

    # Concatenate the positive and negative scores
    scores = torch.cat((pos_scores, neg_scores), dim=1)

    # Average the scores across the embedding dimension
    scores = scores.mean(dim=2)
    scores = F.log_softmax(scores, dim=1)
    target = torch.zeros(batch_size, dtype=torch.long, device=scores.device)

    # Create weight tensor
    weight = torch.ones(num_negatives + 1, device=scores.device)
    weight[0] = pos_weight

    return F.nll_loss(scores, target, weight)

In [ ]:
def margin_ranking_loss(model, claims, pos_evidences, neg_evidences, margin=1.5):
    batch_size = claims.size(0)

    # Get the scores for positive evidence
    pos_scores = model(claims, pos_evidences).unsqueeze(1)

    # Get the scores for negative evidence
    neg_scores_list = [model(claims, neg_evidences[:, i, :]).unsqueeze(1) for i in range(neg_evidences.shape[1])]
    neg_scores = torch.cat(neg_scores_list, dim=1)

    # Calculate the margin ranking loss
    target = torch.ones_like(neg_scores, device=claims.device)
    pos_scores = pos_scores.expand_as(neg_scores)  # Expand pos_scores to match neg_scores shape
    loss = F.margin_ranking_loss(pos_scores, neg_scores, target, margin=margin)

    return loss


# 2.2.1 Ranking Models-Transformer

In [ ]:
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim

class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 512):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(-torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(0)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        token_embedding = token_embedding + self.pos_embedding[:, :token_embedding.size(1), :]
        return self.dropout(token_embedding)

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        embeddings = self.embedding(tokens.long()) * math.sqrt(self.emb_size)
        return embeddings

class TransformerEncoderLayer(nn.Module):
    def __init__(self, emb_size: int, nhead: int, dim_feedforward: int, dropout: float):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(emb_size, nhead, dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(emb_size, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, emb_size)
        self.norm1 = nn.LayerNorm(emb_size)
        self.norm2 = nn.LayerNorm(emb_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src: torch.Tensor, src_key_padding_mask: torch.Tensor):
        src2, _ = self.self_attn(src, src, src, key_padding_mask=src_key_padding_mask)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.gelu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int, nhead: int = 4, num_encoder_layers: int = 3, dim_feedforward: int = 512, dropout: float = 0.5):
        super(TransformerEncoder, self).__init__()
        self.token_embedding = TokenEmbedding(vocab_size, emb_size)
        self.pos_encoder = PositionalEncoding(emb_size, dropout)
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(emb_size, nhead, dim_feedforward, dropout)
            for _ in range(num_encoder_layers)
        ])
        self.layer_norm = nn.LayerNorm(emb_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src: torch.Tensor, src_key_padding_mask: torch.Tensor):
        src = self.token_embedding(src)
        src = self.pos_encoder(src)
        for layer in self.layers:
            src = layer(src, src_key_padding_mask)
        src = self.layer_norm(src)
        return src

class SiameseTransformer(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super(SiameseTransformer, self).__init__()
        self.encoder = TransformerEncoder(vocab_size, emb_size)

    def forward(self, claims: torch.Tensor, evidences: torch.Tensor):
        # Create mask on the fly
        claims_mask = (claims == 0)  # Mask positions where index is 0 (padding)
        evidences_mask = (evidences == 0)  # Mask positions where index is 0 (padding)

        claims_enc = self.encoder(claims, claims_mask)  # Transformer expects key_padding_mask with 1s at padded positions
        evidences_enc = self.encoder(evidences, evidences_mask)  # Transformer expects key_padding_mask with 1s at padded positions

        claims_enc = torch.mean(claims_enc, dim=1)  # Mean pooling over the sequence length
        evidences_enc = torch.mean(evidences_enc, dim=1)  # Mean pooling over the sequence length

        scores_dot = torch.bmm(claims_enc.unsqueeze(1), evidences_enc.unsqueeze(2)).squeeze()  # Dot product
        scores_l2 = -torch.cdist(claims_enc, evidences_enc, p=2)  # Negative L2 distance
        return scores_dot + scores_l2

In [ ]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(-torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(0)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        token_embedding = token_embedding + self.pos_embedding[:, :token_embedding.size(1), :]
        return self.dropout(token_embedding)

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        embeddings = self.embedding(tokens.long()) * math.sqrt(self.emb_size)
        return embeddings

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, nhead=8, num_encoder_layers=3, dim_feedforward=512, dropout=0.5):
        super(TransformerEncoder, self).__init__()
        self.token_embedding = TokenEmbedding(vocab_size, embed_dim)
        self.pos_encoder = PositionalEncoding(embed_dim, dropout)
        self.layer_norm = nn.LayerNorm(embed_dim)
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_encoder_layers)
        self.attention = nn.MultiheadAttention(embed_dim, nhead, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(embed_dim, 1)

    def forward(self, src, src_key_padding_mask):
        src = self.token_embedding(src)
        src = self.pos_encoder(src)
        src = self.layer_norm(src)
        output = self.transformer_encoder(src, src_key_padding_mask=src_key_padding_mask)

        # Apply attention mechanism
        attn_output, _ = self.attention(output, output, output, key_padding_mask=src_key_padding_mask)

        # Compute attention weights
        attn_weights = torch.softmax(self.fc(attn_output), dim=1)

        # Compute context vector as a weighted sum of the encoder outputs
        context_vector = torch.sum(attn_weights * attn_output, dim=1)

        return context_vector

class SiameseTransformer(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super(SiameseTransformer, self).__init__()
        self.encoder = TransformerEncoder(vocab_size, emb_size)

    def forward(self, claims: torch.Tensor, evidences: torch.Tensor):
        # Create mask on the fly
        claims_mask = (claims == 0)  # Mask positions where index is 0 (padding)
        evidences_mask = (evidences == 0)  # Mask positions where index is 0 (padding)

        claims_enc = self.encoder(claims, claims_mask)  # Transformer expects key_padding_mask with 1s at padded positions
        evidences_enc = self.encoder(evidences, evidences_mask)  # Transformer expects key_padding_mask with 1s at padded positions

        # Ensure the encodings have the correct dimensions for the operations
        claims_enc = claims_enc.unsqueeze(1)  # Shape: (batch_size, 1, embed_dim)
        evidences_enc = evidences_enc.unsqueeze(2)  # Shape: (batch_size, embed_dim, 1)

        # Compute dot product and L2 distance
        scores_dot = torch.bmm(claims_enc, evidences_enc).squeeze()  # Dot product
        scores_l2 = -torch.cdist(claims_enc.squeeze(1), evidences_enc.squeeze(2), p=2)  # Negative L2 distance


        return scores_dot + scores_l2


# Binary Transformer

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(-torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(0)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        token_embedding = token_embedding + self.pos_embedding[:, :token_embedding.size(1), :]
        return self.dropout(token_embedding)

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        embeddings = self.embedding(tokens.long()) * math.sqrt(self.emb_size)
        return embeddings

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, nhead=8, num_encoder_layers=2, dim_feedforward=1024, dropout=0.3):
        super(TransformerEncoder, self).__init__()
        self.token_embedding = TokenEmbedding(vocab_size, embed_dim)
        self.pos_encoder = PositionalEncoding(embed_dim, dropout)
        self.layer_norm = nn.LayerNorm(embed_dim)
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_encoder_layers)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_key_padding_mask):
        src = self.token_embedding(src)
        src = self.pos_encoder(src)
        src = self.layer_norm(src)
        output = self.transformer_encoder(src, src_key_padding_mask=src_key_padding_mask)
        return output

class SiameseTransformer(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super(SiameseTransformer, self).__init__()
        self.encoder = TransformerEncoder(vocab_size, emb_size)
        self.fc = nn.Linear(emb_size, 1)  # Binary classification output

    def forward(self, sequences: torch.Tensor):
        # Create mask on the fly
        mask = (sequences == 0)  # Mask positions where index is 0 (padding)

        encodings = self.encoder(sequences, mask)  # Transformer expects key_padding_mask with 1s at padded positions

        # Extract the [CLS] token representation (first token)
        cls_encoding = encodings[:, 0, :]  # Take the [CLS] token representation

        scores = self.fc(cls_encoding)  # Apply the linear layer for binary classification
        return scores.squeeze() # Squeeze the output to get a 1D tensor


# 2.2.2 GRU+Attention

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_dim):
        super(SiameseNetwork, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.gru = nn.GRU(embed_size, hidden_dim, batch_first=True, bidirectional=True)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)

    def attention_net(self, gru_output, mask):
        attn_weights = self.attention(gru_output)  # (batch_size, seq_len, 1)
        attn_weights = attn_weights.squeeze(2)  # (batch_size, seq_len)

        # Apply mask - set attention weights to a large negative value (-inf) where mask is 0
        attn_weights.masked_fill_(mask == 0, float('-inf'))

        attn_weights = torch.softmax(attn_weights, dim=1)  # (batch_size, seq_len)
        attn_weights = attn_weights.unsqueeze(2)  # (batch_size, seq_len, 1)

        context = torch.bmm(attn_weights.transpose(1, 2), gru_output)  # (batch_size, 1, hidden_dim * 2)
        context = context.squeeze(1)  # (batch_size, hidden_dim * 2)
        return context

    def forward_one(self, x):
        x_emb = self.embedding(x)
        gru_output, hidden = self.gru(x_emb)

        mask = (x != 0).float()  # (batch_size, seq_len)

        attn_output = self.attention_net(gru_output, mask)  # (batch_size, hidden_dim * 2)
        output = F.relu(self.fc(attn_output))  # (batch_size, hidden_dim)
        return output

    def forward(self, claims, evidences):
        claims_enc = self.forward_one(claims)
        evidences_enc = self.forward_one(evidences)

        scores_dot = torch.bmm(claims_enc.unsqueeze(1), evidences_enc.unsqueeze(2)).squeeze()  # Dot product
        scores_l2 = -torch.cdist(claims_enc, evidences_enc, p=2)  # Negative L2 distance
        return scores_dot + scores_l2

# 2.3 Model Training

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vocab_size = len(vocab)
embed_dim = 300
hidden_dim = 512
dataset = RankingDataset(train_claims_text_processed, evidence_text_processed, train_evidence_idxs, train_top_indices)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)


#model = SiameseTransformer(vocab_size, embed_dim)
model_path = '/content/drive/MyDrive/nlp/data/best_model.pth'
model = SiameseNetwork(vocab_size, embed_dim, hidden_dim)
model = model.to(device)
criterion = margin_ranking_loss#listwise_loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

clip_value = 1.0

num_epochs = 30

def evaluate_model(model, claims, evidence_texts, top_k, vocab, pad_idx):
    dev_top_indices = topk_indices(dev_reranked_indices, k=top_k)
    #dev_top_scores = topk_indices(dev_reranked_scores, k=top_k)

    dev_scores = []
    for idx in range(len(claims)):
        top_k_evidence_idxs = dev_top_indices[idx]
        top_k_evidences = [evidence_texts[i] for i in top_k_evidence_idxs]
        scores = score_query(model, claims[idx], top_k_evidences, vocab, pad_idx)
        dev_scores.append(scores)

    reranked_indices = []
    for indices, scores in zip(dev_top_indices, dev_scores):
        indexed_scores = list(zip(indices, scores))
        sorted_by_score = sorted(indexed_scores, key=lambda x: x[1], reverse=True)
        sorted_indices = [idx for idx, _ in sorted_by_score]
        reranked_indices.append(sorted_indices)

    return reranked_indices

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    best_f1 = 0.0
    for batch in dataloader:
        if batch is None:
            continue

        claim, pos_evidences, neg_evidences = batch
        claim = claim.to(device)
        pos_evidences = pos_evidences.to(device)
        neg_evidences = neg_evidences.to(device)

        optimizer.zero_grad()
        loss = criterion(model, claim, pos_evidences, neg_evidences)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()
        epoch_loss += loss.item()

    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_epoch_loss}')

    scheduler.step(avg_epoch_loss)
    current_lr = scheduler.optimizer.param_groups[0]['lr']
    print(f"Current Learning Rate: {current_lr}")

    # Evaluation at the end of each epoch
    model.eval()
    with torch.no_grad():
        reranked_indices = evaluate_model(model, dev_claims_text_precessed, evidence_text_processed, top_k=15, vocab=vocab, pad_idx=vocab['<pad>'])
        results = evaluate_evidence_retrieval(reranked_indices, dev_evidence_idxs, k=5)
        f1 = results['average_fscore']
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), model_path)
        print(f"Epoch {epoch+1} Evaluation - Recall: {results['average_recall']}, Precision: {results['average_precision']}, F1 Score: {results['average_fscore']}")


Epoch 1/30, Average Loss: 0.8608104250369928
Current Learning Rate: 0.001
Epoch 1 Evaluation - Recall: 0.131060606060606, Precision: 0.07142857142857141, F1 Score: 0.08640486497629353
Epoch 2/30, Average Loss: 0.4559433765900441
Current Learning Rate: 0.001
Epoch 2 Evaluation - Recall: 0.13755411255411254, Precision: 0.07012987012987013, F1 Score: 0.08547722119150691
Epoch 3/30, Average Loss: 0.31614012404894215
Current Learning Rate: 0.001
Epoch 3 Evaluation - Recall: 0.1362554112554112, Precision: 0.07012987012987013, F1 Score: 0.08520923520923518
Epoch 4/30, Average Loss: 0.2520669845816417
Current Learning Rate: 0.001
Epoch 4 Evaluation - Recall: 0.11493506493506486, Precision: 0.06493506493506494, F1 Score: 0.07641207998350856
Epoch 5/30, Average Loss: 0.18203756385124648
Current Learning Rate: 0.001
Epoch 5 Evaluation - Recall: 0.14588744588744582, Precision: 0.07402597402597398, F1 Score: 0.09015151515151512
Epoch 6/30, Average Loss: 0.14629715021986228
Current Learning Rate: 0.

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torch.nn.utils
import torch.optim.lr_scheduler as lr_scheduler

def binary_classification_loss(model, sequences, labels):
    outputs = model(sequences)
    loss = F.binary_cross_entropy_with_logits(outputs, labels)
    return loss

def train(model, dataloader, optimizer, scheduler, epochs=10, clip_value=1.0):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for sequences_tensor, labels_tensor in dataloader:
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Compute the loss
            loss = binary_classification_loss(model, sequences_tensor, labels_tensor)
            # Backpropagate the loss
            loss.backward()

            # Clip the gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)

            # Update the weights
            optimizer.step()
            # Accumulate the loss for this epoch
            epoch_loss += loss.item()

        # Adjust the learning rate based on the epoch loss
        scheduler.step(epoch_loss)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(dataloader):.4f}")

# Initialize the dataset and dataloader
dataset = RankingDataset(train_claims_text_processed, evidence_text_processed, train_evidence_idxs, train_top_indices, vocab)
dataloader = DataLoader(dataset, batch_size=32, collate_fn=collate_fn, shuffle=True)

# Initialize the model, optimizer, and training
model = SiameseTransformer(len(vocab), emb_size=320).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Initialize the learning rate scheduler
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
train(model, dataloader, optimizer, scheduler, epochs=30)




Epoch 1/30, Loss: 0.3572
Epoch 2/30, Loss: 0.3489
Epoch 3/30, Loss: 0.3484
Epoch 4/30, Loss: 0.3483
Epoch 5/30, Loss: 0.3481
Epoch 6/30, Loss: 0.3478
Epoch 7/30, Loss: 0.3480
Epoch 8/30, Loss: 0.3480
Epoch 9/30, Loss: 0.3479
Epoch 10/30, Loss: 0.3473
Epoch 11/30, Loss: 0.3469
Epoch 12/30, Loss: 0.3458
Epoch 13/30, Loss: 0.3463
Epoch 14/30, Loss: 0.3458
Epoch 15/30, Loss: 0.3465
Epoch 16/30, Loss: 0.3457
Epoch 17/30, Loss: 0.3463
Epoch 18/30, Loss: 0.3456


KeyboardInterrupt: 

ERROR: unknown command "PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True"


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
from torch.nn.utils.rnn import pad_sequence
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dev_top_indices = topk_indices(dev_reranked_indices, k=10)
dev_top_scores = topk_indices(dev_reranked_scores, k=10)
#dev_top_indices, dev_orig_scores = get_top_k_indices(dev_similarity_scores, k=50)
def text_to_indices(text, vocab):
    indices = [vocab["<sos>"]] + [vocab.get(word, vocab["<unk>"]) for word in text] + [vocab["<eos>"]]
    return indices


def create_mask(tensor, pad_idx=vocab['<pad>']):
    return (tensor == pad_idx).to(tensor.device)
def score_query(model, query, evidences, vocab, pad_idx):
    query_indices = text_to_indices(query, vocab)
    evidence_indices = [text_to_indices(evidence, vocab) for evidence in evidences]

    query_tensor = pad_sequence([torch.tensor(query_indices)], batch_first=True, padding_value=pad_idx)
    evidence_tensors = pad_sequence([torch.tensor(ei) for ei in evidence_indices], batch_first=True, padding_value=pad_idx)

    # query_mask = create_mask(query_tensor, pad_idx).squeeze(0)
    # evidence_masks = create_mask(evidence_tensors, pad_idx)

    query_tensor = query_tensor.to(device)
    evidence_tensors = evidence_tensors.to(device)
    # query_mask = query_mask.to(device)
    # evidence_masks = evidence_masks.to(device)

    model.eval()
    scores = []
    with torch.no_grad():
        for i in range(evidence_tensors.shape[0]):
            evidence_tensor = evidence_tensors[i].unsqueeze(0)
            #evidence_mask = evidence_masks[i].unsqueeze(0)
            score = model(query_tensor, evidence_tensor)# query_mask.unsqueeze(0), evidence_mask
            scores.append(score.item())

    return scores

dev_scores = []
for idx in range(len(dev_claims_text_precessed)):
    top_k_evidence_idxs = dev_top_indices[idx]
    top_k_evidences = [evidence_text_processed[i] for i in top_k_evidence_idxs]
    scores = score_query(model, dev_claims_text_precessed[idx], top_k_evidences, vocab, vocab['<pad>'])
    dev_scores.append(scores)


KeyboardInterrupt: 

In [ ]:
from torch.nn.utils.rnn import pad_sequence
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def text_to_indices(text, vocab):
#     indices = [vocab["<cls>"]] + [vocab.get(word, vocab["<unk>"]) for word in text]
#     return indices
dev_top_indices = topk_indices(dev_reranked_indices, k=20)
dev_top_scores = topk_indices(dev_reranked_scores, k=20)

def score_query(model, query, evidences, vocab, pad_idx):
    model = model.to(device)
    model.eval()

    query_indices = text_to_indices(query, vocab)
    probabilities = []

    with torch.no_grad():
        for evidence in evidences:
            evidence_indices = text_to_indices(evidence, vocab)
            combined_indices = [vocab["<cls>"]] + query_indices + [vocab["<sep>"]] + evidence_indices + [vocab["<sep>"]]

            combined_tensor = pad_sequence([torch.tensor(combined_indices, dtype=torch.long)], batch_first=True, padding_value=pad_idx)
            combined_tensor = combined_tensor.to(device)

            logits = model(combined_tensor)
            prob = torch.sigmoid(logits).item()  # Apply sigmoid to get probability
            probabilities.append(prob)

    return probabilities

# Example usage for evaluation
dev_scores = []
for idx in range(len(dev_claims_text_precessed)):
    top_k_evidence_idxs = dev_top_indices[idx]
    top_k_evidences = [evidence_text_processed[i] for i in top_k_evidence_idxs]
    scores = score_query(model, dev_claims_text_precessed[idx], top_k_evidences, vocab, vocab['<pad>'])
    dev_scores.append(scores)


In [ ]:
dev_scores[0]

[0.10233554989099503,
 0.059117142111063004,
 0.06067594140768051,
 0.08072149008512497,
 0.0576542466878891,
 0.09556568413972855,
 0.08637840300798416,
 0.06461432576179504,
 0.06730613112449646,
 0.05402364209294319,
 0.06346571445465088,
 0.06596881151199341,
 0.08334793895483017,
 0.06876970082521439,
 0.11062392592430115,
 0.05371873825788498,
 0.05371873825788498,
 0.06277293711900711,
 0.0736880674958229,
 0.06580596417188644,
 0.07230368256568909,
 0.06807564944028854,
 0.05645347386598587,
 0.06784305721521378,
 0.05965680629014969,
 0.08406560868024826,
 0.07724510878324509,
 0.056744612753391266,
 0.05356026440858841,
 0.05608903616666794,
 0.05610428750514984,
 0.05978047847747803,
 0.05610302463173866,
 0.07313989102840424,
 0.062173787504434586,
 0.058304812759160995,
 0.054597407579422,
 0.062362659722566605,
 0.08891953527927399,
 0.062082018703222275,
 0.06124025583267212,
 0.1053486317396164,
 0.06375909596681595,
 0.0992824137210846,
 0.06311468034982681,
 0.0828681

In [ ]:
def aggregate_scores(original_scores, new_scores, lam=0.7):
    """
    Aggregate scores from two different sources using a weighted sum approach.

    Parameters:
    - original_scores: List of lists containing scores from the original model.
    - new_scores: List of lists containing scores from the new model.
    - lam: Weighting factor for new_scores; 1 - lam will be the weight for the original_scores.

    Returns:
    - List of lists containing aggregated scores.
    """
    aggregated_scores = []
    for original, new in zip(original_scores, new_scores):
        # Calculate the weighted sum of scores
        aggregated = [(1 - lam) * o + lam * n for o, n in zip(original, new)]
        aggregated_scores.append(aggregated)
    return aggregated_scores

scores = aggregate_scores(dev_top_scores, dev_scores)

In [ ]:
reranked_indices = []

for indices, scores in zip(dev_top_indices, dev_scores):
    # Combine indices and scores into a list of tuples
    indexed_scores = list(zip(indices, scores))

    # Sort the list of tuples by the score in descending order
    sorted_by_score = sorted(indexed_scores, key=lambda x: x[1], reverse=True)

    # Extract the sorted indices
    sorted_indices = [idx for idx, _ in sorted_by_score]

    # Add to the final list
    reranked_indices.append(sorted_indices)

In [ ]:
def evaluate_evidence_retrieval(predicted_indices_list, actual_indices_list, k=5):
    #assert len(predicted_indices_list) == len(actual_indices_list), "Both inputs must have the same length."

    total_recall = 0.0
    total_precision = 0.0
    total_fscore = 0.0
    num_claims = len(predicted_indices_list)

    for predicted_indices, actual_indices in zip(predicted_indices_list, actual_indices_list):
        # Convert tensors in predicted_indices to integers if they are not already
        predicted_indices = [index.item() if isinstance(index, torch.Tensor) else index for index in predicted_indices]

        # Retrieve the top k predictions
        top_k_predicted = set(predicted_indices[:k])
        actual_indices_set = set(actual_indices)

        # Calculate the number of correct predictions
        correct_predictions = len(top_k_predicted.intersection(actual_indices_set))

        # Calculate metrics
        if correct_predictions > 0:
            recall = float(correct_predictions) / len(actual_indices_set)
            precision = float(correct_predictions) / k
            if (precision + recall) != 0:
                fscore = 2 * (precision * recall) / (precision + recall)
            else:
                fscore = 0.0
        else:
            recall = 0.0
            precision = 0.0
            fscore = 0.0

        # Accumulate the metrics to calculate averages later
        total_recall += recall
        total_precision += precision
        total_fscore += fscore

    # Calculate average metrics
    average_recall = total_recall / num_claims
    average_precision = total_precision / num_claims
    average_fscore = total_fscore / num_claims

    return {
        "average_recall": average_recall,
        "average_precision": average_precision,
        "average_fscore": average_fscore
    }

# Example usage
results = evaluate_evidence_retrieval(reranked_indices, dev_evidence_idxs)
print(results)

{'average_recall': 0.1841991341991341, 'average_precision': 0.10129870129870122, 'average_fscore': 0.12142857142857143}


In [ ]:
test_claims = pd.read_json('/content/drive/MyDrive/nlp/data/test-claims-unlabelled.json', orient='index')
test_claims.reset_index(inplace=True)
test_claims.columns = ['claim_id', 'claim_text']
test_claims_id = test_claims['claim_id']
test_claims_text = test_claims['claim_text']

test_claims_text_processed = test_claims_text.apply(preprocess_data)

test_bm25_results = []
for query in test_claims_text_processed:
    scores = bm25_scores(query, inverted_index, idf_values, avg_doc_length)
    test_bm25_results.append(scores)


test_reranked_indices = [[doc_id for doc_id, _ in sorted(scores.items(), key=lambda x: x[1], reverse=True)] for scores in test_bm25_results]
test_reranked_scores = [[score for _, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)] for scores in test_bm25_results]

test_top_indices = topk_indices(test_reranked_indices, k=50)
test_top_scores = topk_indices(test_reranked_scores, k=50)

In [ ]:
json.dump(reranked_indices, open("/content/drive/MyDrive/nlp/data/reranked_indices.json", "w"))

In [ ]:

test_scores = []
for idx in range(len(test_claims_text_processed)):
    top_k_evidence_idxs = test_top_indices[idx]
    top_k_evidences = [evidence_text_processed[i] for i in top_k_evidence_idxs]
    scores = score_query(model, test_claims_text_processed[idx], top_k_evidences, vocab, vocab['<pad>'])
    test_scores.append(scores)

In [ ]:
#test_scores_agg = aggregate_scores(dev_top_scores, dev_scores)

test_reranked_indices = []

for indices, scores in zip(test_top_indices, test_scores):
    # Combine indices and scores into a list of tuples
    indexed_scores = list(zip(indices, scores))

    # Sort the list of tuples by the score in descending order
    sorted_by_score = sorted(indexed_scores, key=lambda x: x[1], reverse=True)

    # Extract the sorted indices
    sorted_indices = [idx for idx, _ in sorted_by_score]

    # Add to the final list
    test_reranked_indices.append(sorted_indices)

In [ ]:
test_indices_pred = topk_indices(test_reranked_indices, k=5)

In [ ]:
json.dump(test_reranked_indices, open("/content/drive/MyDrive/nlp/data/test_reranked_indices.json", "w"))

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*